# Introduction

The purpose of this notebook is to build CNN's for digit classification using the famous MNIST dataset. This is one of those classical examples in computer vision that is important in its own right but also incredibly interesting to learn from. Applications of such algorithms include reading bank cheques for example. It is therefore clear why we need to achieve incredibly high accuracy in our algorithms.  

# Libraries

In [ ]:
# Core
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
import matplotlib.pyplot as plt
%matplotlib inline
import time
import warnings
warnings.filterwarnings("ignore")

# Sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from keras.utils.vis_utils import plot_model

**Reproducibility**

In [ ]:
# Random seeds
def set_seed(seed=0):
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed()

# Data

In [ ]:
# Training data
train_data=pd.read_csv('../input/digit-recognizer/train.csv')

# Test data (scale to be in [0,1])
test_data=pd.read_csv('../input/digit-recognizer/test.csv')/255

# Dimensions
print('Training dataframe dimensions:',train_data.shape)
print('Test dataframe dimensions:',test_data.shape)

# First 5 entries of training data
train_data.head()

**Check for null values**

In [ ]:
# Missing values values
print('Number of null values in training set:',train_data.isnull().sum().sum())
print('')
print('Number of null values in test set:',test_data.isnull().sum().sum())

**Preview first few images**

In [ ]:
# Figure size
plt.figure(figsize=(9,9))

# Subplot 
for i in range(9):
    img = np.asarray(train_data.iloc[i+18,1:].values.reshape((28,28))/255);
    ax=plt.subplot(3, 3, i+1)
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.title.set_text(f'{train_data.iloc[i+18,0]}')
    plt.imshow(img, cmap='gray')
    
plt.show()

**Explore label distribution**

In [ ]:
# Figure size
plt.figure(figsize=(10,5))

# Countplot
sns.countplot(x='label', data=train_data)
plt.title('Distribution of labels in training set')

**Labels and features**

In [ ]:
# Labels
y=train_data.label

# Scale features to be in [0,1]
X=train_data.drop('label', axis=1)/255

# Delete to save memory
del train_data

**Reshape data**

In [ ]:
# Reshape (-1 means unspecified)
X = X.values.reshape(-1, 28, 28, 1)

# Save test data index for submission
test_index = test_data.index

# Reshape
test_data = test_data.values.reshape(-1, 28, 28, 1)

# Data Augmentation

Data Augmentation is a clever trick that uses your existing training set to create additional 'fake' data by applying small transformations at every epoch. This doesn't change the size of your training set, it just modifies each image slightly at every epoch so the network becomes for robust over many epochs.

**Visualise augmentations**

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range = 18,           # 18 degrees
    zoom_range=0.2,                # 20 %
    fill_mode='constant',          # edge pixels
    cval=0                         # fill value
)

aug = datagen.flow(X[42].reshape(-1, 28, 28, 1))

# Plot using a for loop
fig, axes = plt.subplots(2, 8, figsize = (15, 4))
plt.suptitle('Random Rotation & Random Zoom', fontsize=20, y=1.05)
axes = axes.ravel()

for i in range(16):
    
    aug_img = next(aug)[0]
    axes[i].imshow(aug_img, cmap = 'gray')
    axes[i].axis('off')
    
plt.show()

We'll actually use keras' preprocessing library instead of ImageDataGenerator. This is because the latter works on a CPU whereas the former works on a GPU so this will be faster.

# Model

In [ ]:
# Define model
def build_model():
    model = keras.Sequential([

        # Data Augmentation
        preprocessing.RandomRotation(factor=0.05, fill_mode='constant'),     # 18 degrees
        preprocessing.RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2), fill_mode='constant'),  # 20%

        # Convolutional layer 1
        layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', input_shape=[28,28,1], activation='relu'),
        layers.MaxPool2D(pool_size=2, padding='same'),
        layers.Dropout(rate=0.3),

        # Convolutional layer 2
        layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'),
        layers.MaxPool2D(pool_size=4, padding='same'),
        layers.Dropout(rate=0.3),
        layers.Flatten(),

        # Hidden layer 3
        layers.Dense(units=256, activation='relu'),
        layers.Dropout(rate=0.4),

        # Output layer (softmax returns a probability distribution)
        layers.Dense(units=10, activation='softmax')
    ])

    # Define optimizer, loss function and accuracy metric
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])
    
    return model

**Early stopping**

In [ ]:
# Define early stopping callback on validation loss
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

# Cross validation

In [ ]:
FOLDS = 10
EPOCHS = 200
BATCH_SIZE = 500

test_preds = np.zeros((1, 1))
scores = []
times = []
history_df = pd.DataFrame()

cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    # Start timer
    start = time.time()
    
    # get training and validation sets
    X_train, X_valid = X[train_idx], X[val_idx]
    y_train, y_valid = y[train_idx], y[val_idx]

    # Build and train model
    model = build_model()
    fold_history = model.fit(
        X_train,
        y_train,
        validation_data=(X_valid, y_valid),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        #callbacks=[early_stopping, reduce_lr],
        verbose=False
    )
    
    history_df = history_df.append(pd.DataFrame(fold_history.history))
    
    # Make predictions and measure accuracy
    y_pred = np.argmax(model.predict(X_valid), axis=1)
    score = accuracy_score(y_valid, y_pred)
    scores.append(score)
    
    # Store predictions
    test_preds = test_preds + model.predict(test_data)
    
    # Stop timer
    stop = time.time()
    times.append((stop - start)/60)
    
    # Print accuracy and time
    print(f'Fold {fold} - Accuracy: {score}, Time: {round((stop - start)/60,1)} mins')
    
print('')
print(f'Mean Accuracy: {np.mean(scores)}')

**Plot fold accuracy**

In [ ]:
# Figure size
plt.figure(figsize=(10,5))

# Line plot
plt.plot(scores, label='scores')
plt.plot(np.mean(scores)*np.ones(len(scores)), c='black', label='mean')

# Aesthetics
plt.title('Fold accuracy')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.ylim([0.99,1])
plt.legend()

**Plot fold time**

In [ ]:
# Figure size
plt.figure(figsize=(10,5))

# Line plot
plt.plot(times, label='times', c='orange')
plt.plot(np.mean(times)*np.ones(len(times)), c='black', label='mean')

# Aesthetics
plt.title('Fold time')
plt.xlabel('Fold')
plt.ylabel('Time (mins)')
plt.legend()

**Plot learning curves**

In [ ]:
for i in range(FOLDS):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    plt.suptitle('Fold '+str(i+1), fontsize=20)
    
    plt.subplot(1,2,1)
    ax=history_df.reset_index().loc[EPOCHS*i+1:EPOCHS*(i+1)-1, ['loss', 'val_loss']].plot(title="Cross-entropy", ax=axes[0])
    plt.xlabel('Epoch')
    
    plt.subplot(1,2,2)
    ax=history_df.reset_index().loc[EPOCHS*i+1:EPOCHS*(i+1)-1, ['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot(title="Accuracy", ax=axes[1])
    plt.xlabel('Epoch')

**Soft voting**

In [ ]:
# Soft voting to ensemble predictions
test_preds = np.argmax(test_preds, axis=1)

**Plot predictions**

In [ ]:
# Plot some model predictions
plt.figure(figsize=(15,3.5))
plt.suptitle('Model predictions', fontsize=20, y=1.05)

# Subplot
for i in range(20):
    img = test_data[i];
    ax=plt.subplot(2, 10, i+1)
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.title.set_text(f'Pred:{test_preds[i]}')
    plt.imshow(img, cmap='gray')
    
plt.show()

# Submission

In [ ]:
# Save predictions to file
output = pd.DataFrame({'ImageId': test_index+1,
                       'Label': test_preds})

# Check format
output.head()

In [ ]:
output.to_csv('submission.csv', index=False)